In [23]:
import numpy as np
import glob
import json


In [38]:
from torchvision import transforms
from pathlib import Path
import torch.nn.functional as F
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.plugins import DDPPlugin
import pytorch_lightning as pl
import torchvision.utils as vutils
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl
from typing import List, Callable, Union, Any, TypeVar, Tuple, Optional
# from torch import tensor as Tensor

Tensor = TypeVar('torch.tensor')

In [9]:
ena_local = 'C:/temp/ena/images/train5/'
images = [os.path.split(i)[1] for i in glob.glob(ena_local + '/*.jpg', recursive=True)]

In [4]:
data = np.load('c:/temp/numpyfinal.npy')

In [13]:
metadata_path = 'C:\Projects\wild\data\ENA24\ena24_public.json'
with open(metadata_path) as f:
    d = json.load(f)

In [18]:
cats = {l['image_id']:  l['category_id'] for l in d['annotations'] if l['image_id'] in [i.split('.jpg')[0] for i in images]}
feats = {image.split('.jpg')[0] : feat for image, feat in zip(images, data)}

In [50]:
class ENA6(Dataset):
    def __init__(self, ids, cats, feats, transform):
        self.ids = ids
        self.cats = cats
        self.feats = feats
        self.transforms = transform

    def __getitem__(self, index):
        feats, cats = self.feats.get(index), self.cats.get(index)
        feats, cats = self.transform(feats), self.transform(cats)
        return feats, cats

    def __len__(self):
        return len(self.ids)

In [30]:
class ConvNet1D(nn.Module):
    def __init__(self):
        super().__init__()  
        self.layer1 = nn.Sequential(
            nn.Conv1d(6, 1, 3),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool1d(5))
        self.layer3 = nn.Sequential(
            nn.Linear(70,50),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(50,22),
            nn.Softmax())

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

    def loss_function(self, *args):
        lossF = nn.CrossEntropyLoss()
        preds = args[0]
        true = args[1]
        loss = lossF(preds, true)
        return {'loss': loss}

In [43]:
class Experiment(pl.LightningModule):

    def __init__(self,
                 model: ConvNet1D,
                 params: dict) -> None:
        super(Experiment, self).__init__()

        self.model = model
        self.params = params
        self.curr_device = None
        self.hold_graph = False
        try:
            self.hold_graph = self.params['retain_first_backpass']
        except:
            pass

    def forward(self, input: Tensor, **kwargs) -> Tensor:
        return self.model(input, **kwargs)

    def training_step(self, batch, batch_idx, optimizer_idx = 0):
        real_img, labels = batch
        self.curr_device = real_img.device

        results = self.forward(real_img, labels = labels)
        train_loss = self.model.loss_function(results, labels)

        self.log_dict({key: val.item() for key, val in train_loss.items()}, sync_dist=True)

        return train_loss['loss']

    def validation_step(self, batch, batch_idx, optimizer_idx = 0):
        real_img, labels = batch
        self.curr_device = real_img.device

        results = self.forward(real_img, labels = labels)
        val_loss = self.model.loss_function(results, labels)

        self.log_dict({f"val_{key}": val.item() for key, val in val_loss.items()}, sync_dist=True)


    def configure_optimizers(self):

        optims = []
        scheds = []

        optimizer = optim.Adam(self.model.parameters(),
                               lr=self.params['LR'],
                               weight_decay=self.params['weight_decay'])
        optims.append(optimizer)
        # Check if more than 1 optimizer is required (Used for adversarial training)
        try:
            if self.params['LR_2'] is not None:
                optimizer2 = optim.Adam(getattr(self.model,self.params['submodel']).parameters(),
                                        lr=self.params['LR_2'])
                optims.append(optimizer2)
        except:
            pass

        try:
            if self.params['scheduler_gamma'] is not None:
                scheduler = optim.lr_scheduler.ExponentialLR(optims[0],
                                                             gamma = self.params['scheduler_gamma'])
                scheds.append(scheduler)

                # Check if another scheduler is required for the second optimizer
                try:
                    if self.params['scheduler_gamma_2'] is not None:
                        scheduler2 = optim.lr_scheduler.ExponentialLR(optims[1],
                                                                      gamma = self.params['scheduler_gamma_2'])
                        scheds.append(scheduler2)
                except:
                    pass
                return optims, scheds
        except:
            return optims

In [44]:
class featsDataset(pl.LightningDataModule):
    def __init__(
        self,
        ids: list,
        feats: dict,
        cats: dict,
        train_batch_size: int = 8,
        val_batch_size: int = 8,
        # patch_size: Union[int, Sequence[int]] = (256, 256),
        patch_size: int = 64, # Union[int, Sequence[int]] = (256, 256),
        num_workers: int = 0,
        pin_memory: bool = False,
        **kwargs,
    ):
        super().__init__()
        self.ids = ids,
        self.feats = feats,
        self.cats = cats,        
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.patch_size = patch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory

    def setup(self, stage: Optional[str] = None) -> None:
    
        train_transforms = transforms.Compose([transforms.ToTensor()])
        
        val_transforms = transforms.Compose([transforms.ToTensor()])
        
        self.train_dataset = ENA6(
            self.ids,
            self.feats,
            self.cats,
            transform=train_transforms,
        )
        
        # Replace CelebA with your dataset
        self.val_dataset = ENA6(
            self.ids,
            self.feats,
            self.cats,
            transform=train_transforms,
        )

        self.test_dataset = self.val_dataset
        
    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            pin_memory=self.pin_memory,
        )

    def val_dataloader(self) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(
            self.val_dataset,
            batch_size=self.val_batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=self.pin_memory,
        )
    
    def test_dataloader(self):
        return self.val_dataloader()

In [45]:
exp_params = {
  'LR': 0.005,
  'weight_decay': 0.0,
  'scheduler_gamma': 0.95,
  'manual_seed': 1265
}

class trainer_params:
  gpus: 1
  max_epochs: 100

In [46]:
LOG_DIR = './log2/'
tb_logger =  TensorBoardLogger(save_dir=LOG_DIR,
                               name='lightning_logs',)

Path(f"{tb_logger.log_dir}/Samples").mkdir(exist_ok=True, parents=True)
Path(f"{tb_logger.log_dir}/Reconstructions").mkdir(exist_ok=True, parents=True)
model = ConvNet1D()
experiment = Experiment(model,
                          exp_params)

In [47]:
runner = Trainer(callbacks=[
                     LearningRateMonitor(),
                     ModelCheckpoint(save_top_k=2, 
                                     dirpath =os.path.join(LOG_DIR , "checkpoints"), 
                                     monitor= "val_loss",
                                     save_last= True),
                 ],
                 gpus = 1, max_epochs=100)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [51]:
datamodule= featsDataset(ids=[i.split('.jpg')[0] for i in images], feats = data, cats = cats)
datamodule.setup()
runner.fit(experiment, datamodule=datamodule)

AttributeError: 'featsDataset' object has no attribute 'train_dir'